In [1]:
import csv
import random
import datetime
import time
import numpy as np
from nltk.tokenize import word_tokenize

with open('train.tsv') as f:
    trainuser=f.readlines()

with open('test.tsv') as f:
    testuser=f.readlines()

with open('docs.tsv') as f:
    data=f.readlines()



In [3]:
data[0].split('\t')#news id, category, subcategory, title, body

['AA3fgHS',
 'autos',
 'autosnews',
 'First Drive: 2009 Nissan GT-R',
 'See the 2009 Nissan GT-R in action at Nissan\'s test facility in Hokkaido, Japan: First Drive: Part 1 - 2009 Nissan GT-R Video First Drive: Part 2 - 2009 Nissan GT-R Video Scene One. Sendai Raceway, two hours north of Tokyo on the Bullet Train. Sendai is the only public racing circuit where Nissan tested its new GT-R. Apart from the Nurburgring in Germany-around which it\'s the world\'s fastest production car (only the limited-build Porsche Carrera GT laps faster, says Nissan). Sendai is like a mini \'Ring: fast and undulating and full of sweeping corners and heart-in-mouth turns. It\'s a roller-coaster raceway. "We didn\'t test on any other public racetrack because most are too easy," says chief engineer Kazutoshi Mizuno, ex-head of Nissan\'s sports-car racing program. Press the red start button on the center console, and the 3.8-liter V-6 burbles awake, slightly muted by its twin turbos. It\'s a new engine, not a

In [2]:
trainuser[0].split('\t')#impression id, user id, user type, news type, clicked news' dwell time/s, timestamp, clicked samples, unclicked samples, session start, session end

['da04f164-2374-4f9d-8ace-d790933bd799',
 '2B482CDF0A419E1916E923C5FFFFFFFF',
 'A',
 'BBPCEYp BBPClcH BBPD6fE BBQ0LCa BBQG1DH',
 '57 98 46 66 47',
 '11/12/2018 1:29:18 PM#TAB#11/12/2018 1:30:16 PM#TAB#11/12/2018 1:31:55 PM#TAB#11/23/2018 7:25:17 AM#TAB#12/8/2018 1:15:25 PM',
 'BBS7KRx BBS3rAy',
 'BBS7xCn BBQYnJv BBS84oG BBS2YYC BBQRFKa BBS7GgD BBS9qn3 BBS7SDi BBS7LoC BBS9BgN BBS6yA2 BBS8WTT BBS9C6B BBS9Hy6 BBS8VsJ BBS8b6P BBS8kkd BBS7owY BBS7zxt BBS9xtw BBS7yFI BBS7uIF BBS7Kon BBS7Wr4 BBS4s5T BBS5VbF BBR8NJB BBS7Zww BBS86YX BBS84hE BBS7EzE BBS8bk0 BBS9wm9 BBS9nKG BBS58Om BBS7zRe BBRRrXl BBS6gG9 BBS8JfV BBS3D6t BBS8UZe BBS1ow3 BBS0SSm BBRIKPQ BBQsVjC BBS9BS2 BBS9GoG BBS9vfd BBS9suJ BBS9tBY BBS7fpA AAylz6W BBS3OBJ BBRhhYc BBS9BBL BBS5rf9 BBROoYP BBS9rw7 BBRYlEk BBS7DDs BBS9E8M BBS7lNI BBQqesC BBRUBjc BBS3KH4 BBS24s5 BBS8kBZ BBS9fyx BBS8TQp BBPIkbm BBS7PFG BBS8LQy BBS4VuF BBS8sYY',
 '1/12/2019 1:52:14 PM',
 '1/12/2019 2:31:46 PM\n']

In [ ]:
newsindex={'NULL':0}
for i in news:
    newsindex[i]=len(newsindex)

def newsample(x,ratio):
    if ratio >len(x):
        return random.sample(x*(ratio//len(x)+1),ratio)
    else:
        return random.sample(x,ratio)


news={}
category={}
subcategory={}
for i in data:
    line=i.strip().split('\t')
    news[line[0]]=[line[1],line[2],word_tokenize(line[3]),word_tokenize(line[4])]
    category[line[1]]=0
    subcategory[line[2]]=0

In [ ]:
npratio=4
time_T=10
train_sample=[]    
label=[]
user_pos=[]
user_neg=[]


for line in trainuser:
  
    line=line.replace('\n','').split('\t')
  
    clickids=[news[x] for x in line[3].split()][-50:]
    dwelltime=[int(x) for x in line[4].split()][-50:]
    posfeedback=[clickids[x] for x in range(len(dwelltime)) if dwelltime[x] >time_T ]   
    negfeedback=[clickids[x] for x in range(len(dwelltime)) if dwelltime[x] <=time_T ][:20]    
    pdoc=[newsindex[x] for x in line[6].split()]
    ndoc=[newsindex[x] for x in line[7].split()]
    
    for pos in pdoc:
  
        neg_samples=newsample(ndoc,npratio)
        all_samples=neg_samples+[pos]
        concat_label=[0]*npratio+[1]
        train_sample.append(all_samples)
        label.append(concat_label)
        user_pos.append(posfeedback+[0]*(50-len(posfeedback)))
        user_neg.append(negfeedback+[0]*(20-len(negfeedback)))


test_sample=[]    
test_label=[]
test_user_pos=[]
test_user_neg=[]
test_index=[] 
for line in testuser: 
    line=line.replace('\n','').split('\t')
  
    clickids=[newsindex[x] for x in line[3].split()][-50:]
    dwelltime=[int(x) for x in line[4].split()][-50:]
    posfeedback=[clickids[x] for x in range(len(dwelltime)) if dwelltime[x] >time_T ]   
    negfeedback=[clickids[x] for x in range(len(dwelltime)) if dwelltime[x] <=time_T ][:20]    
    pdoc=[newsindex[x] for x in line[6].split()][:50]
    ndoc=[newsindex[x] for x in line[7].split()][:300]
    index=[]
    index.append(len(test_sample)) 
    
    for pos in pdoc:

        test_sample.append(pos)
        test_label.append(1)
        test_user_pos.append(posfeedback+[0]*(50-len(posfeedback)))
        test_user_neg.append(negfeedback+[0]*(20-len(negfeedback)))
    for neg in ndoc:

        test_sample.append(neg)
        test_label.append(0)
        test_user_pos.append(posfeedback+[0]*(50-len(posfeedback)))
        test_user_neg.append(negfeedback+[0]*(20-len(negfeedback)))
    index.append(len(test_sample))
    test_index.append(index)

In [ ]:
word_dict_raw={'PADDING':[0,999999]}

for i in news:
    for j in news[i][2]+news[i][3]:
        if j in word_dict_raw:
            word_dict_raw[j][1]+=1
        else:
            word_dict_raw[j]=[len(word_dict_raw),1] 


word_dict={}
for i in word_dict_raw:
    if word_dict_raw[i][1]>=5:
        word_dict[i]=[len(word_dict),word_dict_raw[i][1]]
print(len(word_dict),len(word_dict_raw))

embdict={}
with open('glove.840B.300d.txt','rb')as f:
    linenb=0
    while True:
        line=f.readline()
        if len(line)==0:
            break
        line = line.split()
        word=line[0].decode()
        linenb+=1
        if len(word) != 0:
            vec=[float(x) for x in line[1:]]
            if word in word_dict:
                embdict[word]=vec




from numpy.linalg import cholesky  

emb_mat=[0]*len(word_dict)
in_dict_emb=[]
for i in embdict.keys():
    emb_mat[word_dict[i][0]]=np.array(embdict[i],dtype='float32')
    in_dict_emb.append(emb_mat[word_dict[i][0]])
in_dict_emb=np.array(in_dict_emb,dtype='float32')

mu=np.mean(in_dict_emb, axis=0)
Sigma=np.cov(in_dict_emb.T)

norm=np.random.multivariate_normal(mu, Sigma, 1)
print(mu.shape,Sigma.shape,norm.shape)

for i in range(len(emb_mat)):
    if type(emb_mat[i])==int:
        emb_mat[i]=np.reshape(norm, 300)
emb_mat[0]=np.zeros(300,dtype='float32')
emb_mat=np.array(emb_mat,dtype='float32')
print(emb_mat.shape)

In [ ]:
news_title=[[0]*30]

for i in news:
    line=[]
    for j in news[i][2]:
        if j in word_dict:
            line.append(word_dict[j][0])
    line=line[:30]
    news_title.append(line+[0]*(30-len(line)))

news_body=[[0]*200]

for i in news:
    line=[]
    for j in news[i][3]:
        if j in word_dict:
            line.append(word_dict[j][0])
    line=line[:200]
    news_body.append(line+[0]*(200-len(line)))

In [ ]:
news_title=np.array(news_title,dtype='int32') 
news_body=np.array(news_body,dtype='int32') 

train_sample=np.array(train_sample,dtype='int32')
label=np.array(label,dtype='int32')
user_pos=np.array(user_pos,dtype='int32')
user_neg=np.array(user_neg,dtype='int32')

test_sample=np.array(test_sample,dtype='int32')
test_label=np.array(test_label,dtype='int32')
test_user_pos=np.array(test_user_pos,dtype='int32')
test_user_neg=np.array(test_user_neg,dtype='int32')

In [ ]:
def generate_batch_data_random(batch_size):
    idx = np.arange(len(label))
    np.random.shuffle(idx)
    y=label
    batches = [idx[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            cand_news_title = news_title[train_sample[i]]
            cand_news_title=[cand_news_title[:,k,:] for k in range(cand_news_title.shape[1])]

            user_title=news_title[user_pos[i]]
            user_title=[user_title[:,k,:] for k in range(user_title.shape[1])]
 
            user_title_neg=news_title[user_neg[i]]
            user_title_neg=[user_title_neg[:,k,:] for k in range(user_title_neg.shape[1])]
      
            cand_news_body = news_body[train_sample[i]]
            cand_news_body=[cand_news_body[:,k,:] for k in range(cand_news_body.shape[1])]
            
            user_body=news_body[user_pos[i]]
            user_body=[user_body[:,k,:] for k in range(user_body.shape[1])]
            
            user_body_neg=news_body[user_neg[i]]
            user_body_neg=[user_body_neg[:,k,:] for k in range(user_body_neg.shape[1])]
             
            yield (cand_news_title+user_title+user_title_neg+cand_news_body+user_body+user_body_neg, [label[i]])

def generate_batch_data_test(batch_size):
    

    idx = np.arange(len(test_label))
    y=test_label
    batches = [idx[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            cand_news_title = news_title[test_sample[i]]
            
            user_title=news_title[test_user_pos[i]]
            user_title=[user_title[:,k,:] for k in range(user_title.shape[1])]
            
            user_title_neg=news_title[test_user_neg[i]]
            user_title_neg=[user_title_neg[:,k,:] for k in range(user_title_neg.shape[1])]
            
            cand_news_body = news_body[test_sample[i]]
            
            user_body=news_body[test_user_pos[i]]
            user_body=[user_body[:,k,:] for k in range(user_body.shape[1])]
            user_body_neg=news_body[test_user_neg[i]]
            user_body_neg=[user_body_neg[:,k,:] for k in range(user_body_neg.shape[1])] 
            
            yield ([cand_news_title]+user_title+user_title_neg+[cand_news_body]+user_body+user_body_neg, [test_label[i]])
        

In [ ]:
def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import keras
from keras.utils.np_utils import to_categorical
from keras.layers import *
from keras.models import Model, load_model
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers 
from keras.optimizers import *

In [6]:

class Attention(Layer):

    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ', 
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK', 
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV', 
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)
        
    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12
                
    def call(self, x):
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1))    
        A = K.softmax(A)
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)

In [ ]:
results=[]
keras.backend.clear_session()

MAX_TITLE_LENGTH=30
MAX_HIS_LEN=50
MAX_NEGHIS_LEN=20
MAX_BODY_LENGTH=200

title_input = Input(shape=(MAX_TITLE_LENGTH,), dtype='int32')
body_input = Input(shape=(MAX_BODY_LENGTH,), dtype='int32')
embedding_layer = Embedding(10, 300,trainable=True)
pos_embedding_layer_title = Embedding(MAX_TITLE_LENGTH+1, 300, trainable=True)(Lambda(lambda x:K.zeros_like(x,dtype='int32')+K.arange(x.shape[1]))(title_input))
pos_embedding_layer_body = Embedding(MAX_BODY_LENGTH+1, 300, trainable=True)(Lambda(lambda x:K.zeros_like(x,dtype='int32')+K.arange(x.shape[1]))(body_input))

embedded_sequences = embedding_layer(title_input)
embedded_sequences = add([embedded_sequences,pos_embedding_layer_title])
embedded_sequences = Dropout(0.2)(embedded_sequences)


word_rep = Attention(8,32)([embedded_sequences]*3)
word_rep=Dropout(0.2)(word_rep)

attention = Dense(200,activation='tanh')(word_rep)
attention = Flatten()(Dense(1)(attention))
attention_weight = Activation('softmax')(attention)
rep=Dot((1, 1))([word_rep, attention_weight])


embedded_sequences2 = embedding_layer(body_input)
embedded_sequences2 = add([embedded_sequences2,pos_embedding_layer_body])
embedded_sequences2 = Dropout(0.2)(embedded_sequences2)

word_rep2 = Attention(8,32)([embedded_sequences2]*3)
word_rep2=Dropout(0.2)(word_rep2)

attention2 = Dense(200,activation='tanh')(word_rep2)
attention2 = Flatten()(Dense(1)(attention2))
attention_weight2 = Activation('softmax')(attention2)
rep2=Dot((1, 1))([word_rep2, attention_weight2])


tb_attention = Dot((1, 2))([rep, word_rep2]) 
tb_attention_weight = Activation('softmax')(tb_attention)
tb_rep=Dot((1, 1))([word_rep2, tb_attention_weight])

bt_attention = Dot((1, 2))([rep2, word_rep]) 
bt_attention_weight = Activation('softmax')(bt_attention)
bt_rep=Dot((1, 1))([word_rep, bt_attention_weight])

news_rep=add([rep,rep2,tb_rep,bt_rep])

news_encoder = Model([title_input,body_input], news_rep)


his_input = [Input((MAX_TITLE_LENGTH,), dtype='int32') for _ in range(MAX_HIS_LEN)]
his_input_body = [Input((MAX_BODY_LENGTH,), dtype='int32') for _ in range(MAX_HIS_LEN)]
his_news_rep = [news_encoder([his_input[_],his_input_body[_]]) for _ in range(MAX_HIS_LEN)]
his_news_rep = concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(doc) for doc in his_news_rep],axis=1)


attention_user = Dense(200,activation='tanh')(his_news_rep)
attention_user = Flatten()(Dense(1)(attention_user))
attention_user_weight = Activation('softmax')(attention_user)
user_rep=keras.layers.Dot((1, 1))([his_news_rep, attention_user_weight])

his_input_neg = [Input((MAX_TITLE_LENGTH,), dtype='int32') for _ in range(MAX_NEGHIS_LEN)]
his_input_neg_body = [Input((MAX_BODY_LENGTH,), dtype='int32') for _ in range(MAX_NEGHIS_LEN)]
his_news_neg_rep =[news_encoder([his_input_neg[_],his_input_neg_body[_]]) for _ in range(MAX_NEGHIS_LEN)]
his_news_neg_rep = concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(doc) for doc in his_news_neg_rep],axis=1)

attention_user_neg = Dense(200,activation='tanh')(his_news_neg_rep)
attention_user_neg = Flatten()(Dense(1)(attention_user_neg))
attention_user_neg_weight = Activation('softmax')(attention_user_neg)
user_rep_neg=keras.layers.Dot((1, 1))([his_news_neg_rep, attention_user_neg_weight])


candidates =[Input((MAX_TITLE_LENGTH,), dtype='int32') for _ in range(1+npratio)]
candidates_body =[Input((MAX_BODY_LENGTH,), dtype='int32') for _ in range(1+npratio)]
candidate_vecs = [news_encoder([candidates[_],candidates_body[_]]) for _ in range(1+npratio)]
candidate_vecs = concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(doc) for doc in candidate_vecs],axis=1)


logits = keras.layers.dot([user_rep, candidate_vecs], axes=-1)
logits_neg = keras.layers.dot([user_rep_neg, candidate_vecs], axes=-1)

pnweight=Dense(1)
logitspn=TimeDistributed(pnweight)(concatenate([Lambda(lambda x :K.expand_dims(x,axis=2))(logits),Lambda(lambda x :K.expand_dims(x,axis=2))(logits_neg)]))
logits_final=Activation('softmax')(Flatten()(logitspn))

model = Model(candidates+his_input+his_input_neg+candidates_body+his_input_body+his_input_neg_body, [logits_final])
model.compile(loss=['categorical_crossentropy'], optimizer=Adam(lr=0.001), metrics=['acc'])



candidate_one = keras.Input((MAX_TITLE_LENGTH,))
candidate_one_body = keras.Input((MAX_BODY_LENGTH,))
candidate_one_vec = news_encoder([candidate_one,candidate_one_body])
        
score = keras.layers.Activation(keras.activations.sigmoid)(
           pnweight(concatenate([dot([user_rep, candidate_one_vec], axes=-1),dot([user_rep_neg, candidate_one_vec], axes=-1)])))
model_test = keras.Model([candidate_one]+his_input+his_input_neg+[candidate_one_body]+his_input_body+his_input_neg_body, score)
from sklearn.metrics import roc_auc_score
for ep in range(2):
    traingen=generate_batch_data_random(28)
    model.fit_generator(traingen, epochs=1,steps_per_epoch=len(label)//28)
    valgen=generate_batch_data_test(30)

    auc=[]
    mrr=[]
    ndcg5=[]
    ndcg10=[]
    pred = model_test.predict_generator(valgen, steps=len(test_sample)//30,verbose=1)
    for m in test_index:
        if np.sum(test_label[m[0]:m[1]])!=0 and m[1]<len(cr):
    
            auc.append(roc_auc_score(test_label[m[0]:m[1]],pred[m[0]:m[1],0]))
            mrr.append(mrr_score(test_label[m[0]:m[1]],pred[m[0]:m[1],0]))
            ndcg5.append(ndcg_score(test_label[m[0]:m[1]],pred[m[0]:m[1],0],k=5))
            ndcg10.append(ndcg_score(test_label[m[0]:m[1]],pred[m[0]:m[1],0],k=10))
    results.append([np.mean(auc),np.mean(mrr),np.mean(ndcg5),np.mean(ndcg10)])
    print(np.mean(auc),np.mean(mrr),np.mean(ndcg5),np.mean(ndcg10))